In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
%load_ext autoreload
%autoreload 2
plt.style.use('ggplot')
import sys, os 
sys.path.append('/'.join(os.getcwd().split('/')[:4]))
from config.get import cfg
from tensorflow import keras
from tensorflow.keras import layers
import autoencoders

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import talos

ImportError: cannot import name 'get_config' from 'tensorflow.python.eager.context' (/ssoft/spack/arvine/v1/opt/spack/linux-rhel7-skylake_avx512/gcc-8.4.0/py-tensorflow-2.3.1-m7w2iqlf7ulhlydlgt2jbpulmb4atxua/lib/python3.7/site-packages/tensorflow/python/eager/context.py)

# loading data

In [2]:
X_train = np.load(cfg['files']["raw_train_features"])
X_test  = np.load(cfg['files']["raw_test_features"])

# Stats on Zero-padding

In [13]:
print(f"Prencentage of padded zero in the training set : {100* np.mean(np.isclose(X_train, 0.0)): 2.2f} %")
print(f"Prencentage of padded zero in the test set     : {100* np.mean(np.isclose(X_test, 0.0)): 2.2f} %")

Prencentage of padded zero in the training set :  17.39 %
Prencentage of padded zero in the test set     :  17.96 %


# loading model

In [34]:
model_name , autoencoder = CNN()
autoencoder.summary()

Model: "functional_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        [(None, 3, 600, 2)]       0         
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 3, 600, 8)         152       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 3, 300, 8)         0         
_________________________________________________________________
conv2d_81 (Conv2D)           (None, 3, 300, 4)         292       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 1, 300, 4)         0         
_________________________________________________________________
conv2d_82 (Conv2D)           (None, 1, 300, 1)         9         
_________________________________________________________________
conv2d_83 (Conv2D)           (None, 1, 300, 4)       

# Training

/!\ does not run yet (talos code reorganisation)

In [6]:
# fit function for talos
def fit_model(X_train, y_train, X_val, y_val, params):
    # build model
    # fit model
    _,autoencoder = autoencoders.Complex_CNN    
    
    out = autoencoder.fit(
        X_train, X_train,
        shuffle=True,
        verbose=1,
        batch_size=params['batch_size'],
        epochs=params['epochs'],
        validation_data=[x_val, y_val],
        callbacks=[talos.utils.early_stopper(params['epochs'])],
    )
    plt.plot(out.history["loss"])
    plt.show()
    return out, autoencoder

In [ ]:
fit_model(autoencoder,X_train, y_train, X_val, y_val, params)

## hyperparameters tuning

In [ ]:
from tensorflow.keras.activations import relu, elu
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten,Dropout
from keras import optimizers
from keras.models import Sequential
from tensorflow.keras import regularizers

## the parameter that you want to be optimized are defined in this dictionnary
p = {
    'activation':['selu', 'elu'],
    'dense_layers' : [2,4,6]
    'first_neuron': [100,200,300],
    'dropout': [0, .25, .5],
    'batch_size': [32],
    'opt': ['adam', 'rmsprop'],
    'weight_regulizer':[None, regularizers.l1(l1=1e-4),  regularizers.l1(l1=1e-2)],
    'epochs': [80,120,160]
}

In [ ]:
scan_object = talos.Scan(
    x=tX_train, y=y_train, params=p, 
    model=fit_model, 
    experiment_name='ml4f-autoencoder', 
) 

In [ ]:
scan_object.evaluate_models(
    x_val=x_val,
    y_val=y_val,
    n_models=10,
    metric='f1score',
    folds=5,
    shuffle=True,
    average='binary',
    asc=False
)

In [ ]:
scan_object.data

In [ ]:
autoencoder.save('autoencoder_model')

# Load and extract encoder

In [5]:
cfg["models"]["autoencoder"]

'/scratch/izar/kapps/DEX-Cyclic-Arbitrage/models/embedding/autoencoder/'

In [1]:
d = {'a': '1', 'b': 'myname'}



'1_myname'

# Model  evaluation

In [19]:
tX_train = np.load(cfg['files']['raw_train_features'])
tX_test  = np.load(cfg['files']['raw_test_features'])
N_train, N_test = tX_train.shape[0], tX_test.shape[0]
N = N_train + N_test

In [53]:
#from sklearn.metrics import mean_squared_error

def mean_squared_error(x,y):
    return np.sum((x-y)**2)/x.size

In [54]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=100)
ttX_train = tX_train.reshape(N_train,-1)
ttX_test = tX_test.reshape(N_test,-1)
pca.fit(ttX_train)
pca_train = mean_squared_error(ttX_train, pca.inverse_transform(pca.transform(ttX_train)))
pca_test =  mean_squared_error(ttX_test, pca.inverse_transform(pca.transform(ttX_test)))
print(f"MSE train : {pca_train} \nMSE test  : {pca_test}")

MSE train : 2701.1471313988195 
MSE test  : 2875.2690935465525


In [64]:
perf = pd.DataFrame({ 'MSE': [35931.47411685519, 34284.88200871408, pca_train, pca_test],'model': ['Autoencoder','Autoencoder', 'PCA','PCA'],'set': ['train set','test set','train set','test set']})

In [73]:
import plotly.express as px
fig = px.bar(perf, x="set", y="MSE",
             color='model', barmode='group',
             height=400, range_y=(2000,40000))
fig.update_layout(title_text="Embedding performance : Autoencoder vs PCA")
fig.update_xaxes(title="")
fig.write_html(f"{cfg['fig_dir']['embedding']}performance_autoencoder.html",full_html=False, include_plotlyjs=False)
fig.show()